In [2]:
# !pip install kplr
# !pip install lightkurve
# !pip install --upgrade tensorflow
# !pip show tensorflow
# !pip install seaborn
# !pip install sklearn
from numpy.random import seed
seed(10)
# from tensorflow import set_random_seed
# import tensorflow
# tensorflow.random.set_seed(10)

In [4]:
# Carga de librerias
# import kplr
import lightkurve as lc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sn
import sklearn

from astropy.io import fits
# from keras.utils import plot_model
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers.convolutional import Conv1D
# from keras.layers.pooling import MaxPooling1D
# from keras.layers.merge import concatenate
from lightkurve import search_targetpixelfile
from lightkurve import search_lightcurvefile
from numpy import sqrt
from numpy import argmax
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score

In [ ]:
# df = pd.read_csv('dataset_koi.csv', sep = ';', header=None, skiprows=1)
# df.head()

df = pd.read_csv("C:/Users/ilaguna/Desktop/Personal/Master-IA/TFM/dataset_koi.csv", sep = ',')
# client = kplr.API()

kepoi_folder = "C:/Users/ilaguna/Desktop/Personal/Master-IA/TFM"
kepoi_file_global = kepoi_folder + "/" + "global.csv"
kepoi_file_local = kepoi_folder + "/" + "local.csv"

if not os.path.exists(kepoi_folder):
  os.makedirs(kepoi_folder)

headers_global = ["KOI_ID", "Label"]
headers_local = ["KOI_ID", "Label"]
for i in range(1, 1002):
  headers_global.append("Flux-" + str(i))
for i in range(1, 102):
  headers_local.append("Flux-" + str(i))

df_global = pd.DataFrame(columns=headers_global)
df_local = pd.DataFrame(columns=headers_local)

# print("--------------------------- PINTAMOS CABECERAS LOCALES ---------------------------")
# print(df_local)
# print(df)

# Lectura de datos y obtención de las curvas de luz para cada exoplaneta
for row in df.index:
  # if not pd.isnull(df.kepler_name[row]): 
  # if df.kepler_name[row] == 'Kepler-227 c':
    print("------------------------------------------------------------------")
    print("row: " + str(row))
    print("Busco las curvas de luz del exoplaneta " + str(df.kepler_name[row]))
    print("------------------------------------------------------------------")

    try:
      # planet_data = client.planet(df.kepler_name[row])
      # period, t0, duration_hours = planet_data.koi_period, planet_data.koi_time0bk, planet_data.koi_duration
      # print("-------------------------------- DATOS POR PLANETA --------------------------------")
      # tmp_period = float(df.koi_period[row].replace(".", ""))
      # period = round((tmp_period/1000000000), 3)
    
      # tmp_t0 = float(df.koi_time0bk[row].replace(".", ""))
      # t0 = round((tmp_t0/10000000), 3)
    
      # duration_hours = round((float(df.koi_duration[row])/100), 3)
    
      # print("period: " + str(period))
      # print("t0: " + str(t0))
      # print("duration_hours: " + str(duration_hours))
      # print("-----------------------------------------------------------------------------------")
    
      print("----- VALORES DE period, t0 y duration -----")
      period = df.koi_period[row]
      t0 = df.koi_time0bk[row]
      duration_hours = df.koi_duration[row]
      print(period)
      print(t0)
      print(duration_hours)
      print("--------------------------------")

      kic = "KIC " + str(df.kepid[row])
      print("[",row,"/", len_df_koi,"] Descargando fits de ", kic, " - ", df.kepler_name[row])
      # lc = lc.search_lightcurvefile(df.kepler_name[row], mission="KEPLER").download_all()
      tmp_lc = lc.search_lightcurvefile(kic, mission='Kepler').download_all()

      # Control para casos con varios targets 
      dict_klc = {}
      for item in tmp_lc.PDCSAP_FLUX:
          if item.targetid not in dict_klc.keys():
              dict_klc[item.targetid] = [item]
          else:
              dict_klc[item.targetid].append(item)

      collection = lc.LightCurveCollection(next(iter(dict_klc.values())))

      # Union de los flujos estelares obtenidos de la curva de luz para generar las imagenes
      # Se utiliza el flujo PDCSAP porque suele ser datos más limpios que el flujo SAP y tendrá menos tendencias sistemáticas.
      lc_stitch = collection.stitch()
      print("Dimensiones de stitch: " + str(lc_stitch.flux.shape))
      # lc_stitch.plot()

      # Eliminacion de anomalías
      lc_cleaned = lc_stitch.remove_outliers(sigma=20, sigma_upper=4)
      # lc_cleaned.plot()

      # Pliegue de la curva de luz para un periodo y t0 (hora central del transito) específicos
      temp_fold = lc_cleaned.fold(period, t0=t0)
      fractional_duration = (duration_hours / 24.0) / period
      phase_mask = np.abs(temp_fold.phase) < (fractional_duration * 1.5)
      transit_mask = np.in1d(lc_cleaned.time, temp_fold.time_original[phase_mask])

      # Aplanado de la curva de luz
      lc_flat, trend_lc = lc_cleaned.flatten(return_trend=True, mask=transit_mask)

      # Pliegue de la curva ya aplanada
      lc_fold = lc_flat.fold(period, t0=t0)
      # lc_fold.plot()

      # Se aplica filtro de mediana y normalizacion
      # Cambio del bin, y se elimina normalize, porque ya estaba normalizado
      # lc_global = lc_fold.bin(bins=1001, method='median') - 1
      # lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1
      # print("Dimensiones de global: " + str(lc_global.flux.shape))

      lc_global = lc_fold.bin(bins=1001, method='median').normalize() - 1
      lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1
      print("-> Dimensiones FLUX global", lc_global.flux.shape)

      lc_global.scatter();

      # Calculo de fase
      phase_mask = (lc_fold.phase > -4*fractional_duration) & (lc_fold.phase < 4.0*fractional_duration)
      lc_zoom = lc_fold[phase_mask]

      # Se aplica filtro de mediana y normalizacion
      # Cambio del bin, y se elimina normalize, porque ya estaba normalizado
      # lc_local = lc_zoom.bin(bins=101, method='median') -1
      # lc_local = (lc_local / np.abs(lc_local.flux.min()) ) * 2.0 + 1
      # print("Dimensiones de local: " + str(lc_local.flux.shape))

      lc_local = lc_zoom.bin(bins=101, method='median').normalize() -1
      print(lc_local)
      lc_local = (lc_local / np.abs(lc_local.flux.min()) ) * 2.0 + 1
      print("-> Dimensiones FLUX local", lc_local.flux.shape)

      lc_local.scatter();

      label = "1" if (df.koi_disposition[row] == 'CONFIRMED' and df.koi_pdisposition[row] == 'CANDIDATE') else "0"

      tmp_global = [df.kepid[row], label]
      tmp_local = [df.kepid[row], label]
      array_global = np.concatenate([tmp_global, lc_global.flux])
      array_local = np.concatenate([tmp_local, lc_local.flux])
      df_global.loc[row] = array_global
      df_local.loc[row] = array_local
    
    except Exception:
      print("Error with planet " + str(df.kepler_name[row]))
      pass  


pd.DataFrame(df_global).to_csv(kepoi_file_global)
pd.DataFrame(df_local).to_csv(kepoi_file_local)

print("-------- FIN --------")

------------------------------------------------------------------
row: 0
Busco las curvas de luz del exoplaneta Kepler-227 b
------------------------------------------------------------------
----- VALORES DE period, t0 y duration -----
9.48803557
170.53875
2.9575
--------------------------------
[ 0 / 9564 ] Descargando fits de  KIC 10797460  -  Kepler-227 b
Dimensiones de stitch: (65031,)


c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "
c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "


-> Dimensiones FLUX global (1001,)
-> Dimensiones FLUX local (101,)
------------------------------------------------------------------
row: 1
Busco las curvas de luz del exoplaneta Kepler-227 c
------------------------------------------------------------------
----- VALORES DE period, t0 y duration -----
54.4183827
162.51384
4.507
--------------------------------
[ 1 / 9564 ] Descargando fits de  KIC 10797460  -  Kepler-227 c
Dimensiones de stitch: (65031,)


c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "
c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "


-> Dimensiones FLUX global (1001,)
-> Dimensiones FLUX local (101,)
------------------------------------------------------------------
row: 2
Busco las curvas de luz del exoplaneta nan
------------------------------------------------------------------
----- VALORES DE period, t0 y duration -----
19.89913995
175.850252
1.7822
--------------------------------
[ 2 / 9564 ] Descargando fits de  KIC 10811496  -  nan
Dimensiones de stitch: (51876,)


...

-> Dimensiones FLUX global (1001,)
-> Dimensiones FLUX local (101,)
------------------------------------------------------------------
row: 11
Busco las curvas de luz del exoplaneta Kepler-2 b
------------------------------------------------------------------
----- VALORES DE period, t0 y duration -----
2.2047354169999998
121.35854170000002
3.88864
--------------------------------
[ 11 / 9564 ] Descargando fits de  KIC 10666592  -  Kepler-2 b
Dimensiones de stitch: (65504,)


c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "
c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "


-> Dimensiones FLUX global (1001,)
-> Dimensiones FLUX local (101,)
------------------------------------------------------------------
row: 12
Busco las curvas de luz del exoplaneta Kepler-8 b
------------------------------------------------------------------
----- VALORES DE period, t0 y duration -----
3.5224984289999997
121.1194228
3.19843
--------------------------------
[ 12 / 9564 ] Descargando fits de  KIC 6922244  -  Kepler-8 b
Dimensiones de stitch: (65504,)


c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "
c:\users\ilaguna\appdata\local\programs\python\python38-32\lib\site-packages\lightkurve\lightcurve.py:698: LightkurveWarning: The light curve already appears to be in relative units; `normalize()` will convert the light curve into relative units for a second time, which is probably not what you want.
  warnings.warn("The light curve already appears to be in relative "


-> Dimensiones FLUX global (1001,)
-> Dimensiones FLUX local (101,)
------------------------------------------------------------------
row: 13
Busco las curvas de luz del exoplaneta Kepler-466 c
------------------------------------------------------------------
----- VALORES DE period, t0 y duration -----
3.709214104
133.98318
2.6302
--------------------------------
[ 13 / 9564 ] Descargando fits de  KIC 10984090  -  Kepler-466 c


In [23]:
import pandas as pd
# Generamos parametros estelares para los exoplanetas descargados
df_koi = pd.read_csv("C:/Users/ilaguna/Desktop/dataset_koi.csv", sep = ';')
df_local = pd.read_csv("C:/Users/ilaguna/Desktop/local_data.csv", sep = ';')
df_stellar_parameters = pd.DataFrame(columns = ['KOI_ID', 'Label', 'koi_steff', 'koi_slogg', 'koi_smet', 'koi_srad', 'koi_smass'])

kepoi_folder = "C:/Users/ilaguna/Desktop"
kepoi_file_stellar_parameters = kepoi_folder + "/" + "stellar_parameters.csv"

for row in df_local.index:    
    new_row = [df_local.KOI_ID[row], df_local.Label[row], koi['koi_steff'].values[0], koi['koi_slogg'].values[0], koi['koi_smet'].values[0], koi['koi_srad'].values[0], koi['koi_smass'].values[0]]
    # new_df = pd.DataFrame(new_row, columns = ['KOI_ID', 'Label', 'koi_steff', 'koi_slogg', 'koi_smet', 'koi_srad', 'koi_smass'])
    # df_stellar_parameters = df_stellar_parameters.append(new_df, ignore_index=True)
    df_stellar_parameters.loc[row] = new_row
    
print("-------- FIN --------")
pd.DataFrame(df_stellar_parameters).to_csv(kepoi_file_stellar_parameters)

-------- FIN --------
